# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [38]:
import os
import joblib
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.train.automl import AutoMLConfig
from azureml.core.compute_target import ComputeTargetException
from azureml.contrib.pipeline.steps import ParallelRunStep, ParallelRunConfig

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd

In [39]:
ws = Workspace.from_config()
print(ws.name, ws.resource_group, ws.location, ws.subscription_id, sep = '\n')

hei-aep-vn001-d-we-ml-mlsa-01
hei-aep-vn001-d-we-rg-mlsa-01
westeurope
99149dbb-f505-4eb9-901a-c82ba247986c


In [40]:
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException

# NOTE: update the cluster name to match the existing cluster
# Choose a name for your CPU cluster
amlcompute_cluster_name = "cluster-bank-marketing-1"

# Verify that cluster does not exist already
try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('Found existing cluster, use it.')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2',# for GPU, use "STANDARD_NC6"
                                                           #vm_priority = 'lowpriority', # optional
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 1, timeout_in_minutes = 10)
# For a more detailed view of current AmlCompute status, use get_status().

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [41]:
from azureml.core.dataset import Dataset
# Try to load the dataset from the Workspace. Otherwise, create it from the file
# NOTE: update the key to match the dataset name
found = False
key = "Heart Failure Dataset"
description_text = "Heart Failure DataSet for Udacity Course 2"

if key in ws.datasets.keys(): 
        found = True
        dataset = ws.datasets[key] 

if not found:
        # Create AML Dataset and register it into Workspace
        example_data = 'https://raw.githubusercontent.com/thangtranquang/Azure-Machine-Learning-Engineer-2023/main/data/heart_failure_clinical_records_dataset.csv'
        dataset = Dataset.Tabular.from_delimited_files(example_data)        
        #Register Dataset in Workspace
        dataset = dataset.register(workspace=ws,
                                   name=key,
                                   description=description_text)


df = dataset.to_pandas_dataframe()
df.describe()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
count,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00,299.00
mean,60.83,0.43,581.84,0.42,38.08,0.35,263358.03,1.39,136.63,0.65,0.32,130.26,0.32
std,11.89,0.50,970.29,0.49,11.83,0.48,97804.24,1.03,4.41,0.48,0.47,77.61,0.47
min,40.00,0.00,23.00,0.00,14.00,0.00,25100.00,0.50,113.00,0.00,0.00,4.00,0.00
25%,51.00,0.00,116.50,0.00,30.00,0.00,212500.00,0.90,134.00,0.00,0.00,73.00,0.00
50%,60.00,0.00,250.00,0.00,38.00,0.00,262000.00,1.10,137.00,1.00,0.00,115.00,0.00
75%,70.00,1.00,582.00,1.00,45.00,1.00,303500.00,1.40,140.00,1.00,1.00,203.00,1.00
max,95.00,1.00,7861.00,1.00,80.00,1.00,850000.00,9.40,148.00,1.00,1.00,285.00,1.00


In [42]:
from azureml.core.dataset import Dataset
from sklearn.model_selection import train_test_split


df1 = dataset.to_pandas_dataframe()
train, test = train_test_split(df1, shuffle=True)
train.to_csv('train.csv', index = False)

datastore = ws.get_default_datastore()
datastore.upload_files(files = ['./train.csv'])

train = Dataset.Tabular.from_delimited_files([(datastore, 'train.csv')])

Uploading an estimated of 1 files
Target already exists. Skipping upload for train.csv
Uploaded 0 files


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [43]:
from azureml.train.automl import AutoMLConfig
import logging
import time

# TODO: Put your automl settings here
automl_settings = {
    "n_cross_validations":2,
    "primary_metric":'accuracy',
    "enable_early_stopping":True,
    "experiment_timeout_minutes":45,
    "max_concurrent_iterations":10,
   }

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    task='classification',
    compute_target='cluster-bank-marketing-1',
    blocked_models = ['XGBoostClassifier'],
    training_data=train,
    label_column_name='DEATH_EVENT',
    featurization = 'auto',
    debug_log = "automl_errors.log",
    **automl_settings
    )

In [44]:
from azureml.core.experiment import Experiment
# choose a name for experiment
experiment_name = 'automl_run'
experiment=Experiment(ws, experiment_name)

# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output=True)

Submitting remote run.
No run_configuration provided, running on cluster-bank-marketing-1 with default configuration
Running on remote compute: cluster-bank-marketing-1


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_800ba8b7-cebe-4b50-a30c-0dc610a8f722,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation



Current status: FeaturesGeneration. Generating features for the dataset.
Current status: DatasetCrossValidationSplit. Generating individually featurized CV splits.
Current status: ModelSelection. Beginning model selection.

********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

**********************************************************************************

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [45]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()
remote_run.wait_for_completion(show_output=True)

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_800ba8b7-cebe-4b50-a30c-0dc610a8f722,automl,Completed,Link to Azure Machine Learning studio,Link to Documentation




********************************************************************************************
DATA GUARDRAILS: 

TYPE:         Class balancing detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and all classes are balanced in your training data.
              Learn more about imbalanced data: https://aka.ms/AutomatedMLImbalancedData

********************************************************************************************

TYPE:         Missing feature values imputation
STATUS:       PASSED
DESCRIPTION:  No feature missing values were detected in the training data.
              Learn more about missing value imputation: https://aka.ms/AutomatedMLFeaturization

********************************************************************************************

TYPE:         High cardinality feature detection
STATUS:       PASSED
DESCRIPTION:  Your inputs were analyzed, and no high cardinality features were detected.
              Learn more about high cardinality feat

{'runId': 'AutoML_800ba8b7-cebe-4b50-a30c-0dc610a8f722',
 'target': 'cluster-bank-marketing-1',
 'status': 'Completed',
 'startTimeUtc': '2023-05-28T17:41:49.190839Z',
 'endTimeUtc': '2023-05-28T17:55:24.680554Z',
 'services': {},
 'warnings': [{'source': 'JasmineService',
   'message': 'No scores improved over last 10 iterations, so experiment stopped early. This early stopping behavior can be disabled by setting enable_early_stopping = False in AutoMLConfig for notebook/python SDK runs.'}],
 'properties': {'num_iterations': '1000',
  'training_type': 'TrainFull',
  'acquisition_function': 'EI',
  'primary_metric': 'accuracy',
  'train_split': '0',
  'acquisition_parameter': '0',
  'num_cross_validation': '2',
  'target': 'cluster-bank-marketing-1',
  'AMLSettingsJsonString': '{"path":null,"name":"automl_run","subscription_id":"99149dbb-f505-4eb9-901a-c82ba247986c","resource_group":"hei-aep-vn001-d-we-rg-mlsa-01","workspace_name":"hei-aep-vn001-d-we-ml-mlsa-01","region":"westeurope","

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [46]:
# Retrieve and save your best automl model.
best_remote_run,best_automl_model=remote_run.get_output()
best_run_metrics=remote_run.get_metrics()
for primary_metric in best_run_metrics:
    metric=best_run_metrics[primary_metric]
    print(primary_metric,metric)

Package:azureml-automl-runtime, training version:1.48.0.post2, current version:1.44.0
Package:azureml-core, training version:1.48.0, current version:1.44.0
Package:azureml-dataprep, training version:4.8.6, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.15.2, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.44.0
Package:azureml-defaults, training version:1.48.0, current version:1.44.0
Package:azureml-interpret, training version:1.48.0, current version:1.44.0
Package:azureml-mlflow, training version:1.48.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.44.0
Package:azureml-telemetry, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.48.0.post2, curr

experiment_status ['DatasetEvaluation', 'FeaturesGeneration', 'DatasetFeaturization', 'DatasetFeaturizationCompleted', 'DatasetCrossValidationSplit', 'ModelSelection']
experiment_status_description ['Gathering dataset statistics.', 'Generating features for the dataset.', 'Beginning to fit featurizers and featurize the dataset.', 'Completed fit featurizers and featurizing the dataset.', 'Generating individually featurized CV splits.', 'Beginning model selection.']
AUC_micro 0.9109933035714286
f1_score_weighted 0.8673750557662405
accuracy 0.8705357142857143
precision_score_macro 0.8677399800313793
average_precision_score_micro 0.9098893962247427
recall_score_micro 0.8705357142857143
precision_score_weighted 0.8788235196731666
AUC_weighted 0.911885229714358
f1_score_micro 0.8705357142857143
average_precision_score_weighted 0.916531340910196
matthews_correlation 0.7169061546508154
f1_score_macro 0.850684206298937
balanced_accuracy 0.8507946789686552
log_loss 0.40562709629316196
norm_macro_

In [47]:
#TODO: Save the best model
print(best_remote_run)
best_remote_run

Run(Experiment: automl_run,
Id: AutoML_800ba8b7-cebe-4b50-a30c-0dc610a8f722_48,
Type: azureml.scriptrun,
Status: Completed)


Experiment,Id,Type,Status,Details Page,Docs Page
automl_run,AutoML_800ba8b7-cebe-4b50-a30c-0dc610a8f722_48,azureml.scriptrun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [48]:
#TODO: Save the best model
import pickle
best_remote_run,best_automl_model=remote_run.get_output()
#joblib.dump(value=best_automl_model,filename='automl_model.joblib')
pkl_filename = "automl_model.pkl"
with open(pkl_filename,'wb') as f:
    pickle.dump(best_automl_model,f)

Package:azureml-automl-runtime, training version:1.48.0.post2, current version:1.44.0
Package:azureml-core, training version:1.48.0, current version:1.44.0
Package:azureml-dataprep, training version:4.8.6, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.15.2, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.44.0
Package:azureml-defaults, training version:1.48.0, current version:1.44.0
Package:azureml-interpret, training version:1.48.0, current version:1.44.0
Package:azureml-mlflow, training version:1.48.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.44.0
Package:azureml-telemetry, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.48.0.post2, curr

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

In [49]:
from azureml.core.model import Model
# Register model
model = Model.register(workspace = ws,
                        model_name = "Heart_failure_prediction",
                        model_path = "./automl_model.pkl",
                        description = "Best AutoML Model",)

Registering model Heart_failure_prediction


TODO: In the cell below, send a request to the web service you deployed to test it.

In [50]:
%%writefile amlscore.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("automl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error


Overwriting amlscore.py


In [51]:
best_run_metrics

{'experiment_status': ['DatasetEvaluation',
  'FeaturesGeneration',
  'DatasetFeaturization',
  'DatasetFeaturizationCompleted',
  'DatasetCrossValidationSplit',
  'ModelSelection'],
 'experiment_status_description': ['Gathering dataset statistics.',
  'Generating features for the dataset.',
  'Beginning to fit featurizers and featurize the dataset.',
  'Completed fit featurizers and featurizing the dataset.',
  'Generating individually featurized CV splits.',
  'Beginning model selection.'],
 'AUC_micro': 0.9109933035714286,
 'f1_score_weighted': 0.8673750557662405,
 'accuracy': 0.8705357142857143,
 'precision_score_macro': 0.8677399800313793,
 'average_precision_score_micro': 0.9098893962247427,
 'recall_score_micro': 0.8705357142857143,
 'precision_score_weighted': 0.8788235196731666,
 'AUC_weighted': 0.911885229714358,
 'f1_score_micro': 0.8705357142857143,
 'average_precision_score_weighted': 0.916531340910196,
 'matthews_correlation': 0.7169061546508154,
 'f1_score_macro': 0.8506

In [52]:
from azureml.automl.core.shared import constants
best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, 'azureml-automl.yml')

In [53]:
# create inference_config
from azureml.core.model import InferenceConfig
from azureml.core.environment import Environment

inference_config = InferenceConfig(entry_script="scoring.py")

In [54]:
from azureml.core.webservice import Webservice, AciWebservice
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb = 1)
print(deployment_config)

In [55]:
best_remote_run, model = remote_run.get_output()
best_remote_run_metrics = best_remote_run.get_metrics()

print(model)
print('Best Run Id: ', best_remote_run.id)
print('\n Accuracy:', best_remote_run_metrics['accuracy'])

Package:azureml-automl-runtime, training version:1.48.0.post2, current version:1.44.0
Package:azureml-core, training version:1.48.0, current version:1.44.0
Package:azureml-dataprep, training version:4.8.6, current version:4.2.2
Package:azureml-dataprep-rslex, training version:2.15.2, current version:2.8.1
Package:azureml-dataset-runtime, training version:1.48.0, current version:1.44.0
Package:azureml-defaults, training version:1.48.0, current version:1.44.0
Package:azureml-interpret, training version:1.48.0, current version:1.44.0
Package:azureml-mlflow, training version:1.48.0, current version:1.44.0
Package:azureml-pipeline-core, training version:1.48.0, current version:1.44.0
Package:azureml-responsibleai, training version:1.48.0, current version:1.44.0
Package:azureml-telemetry, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-client, training version:1.48.0, current version:1.44.0
Package:azureml-train-automl-runtime, training version:1.48.0.post2, curr

Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mnt/batch/tasks/shared/LS_root/mount...
                                                                            ('logisticregression',
                                                                             LogisticRegression(C=51.79474679231202,
                                                                                                class_weight=None,
                                                                                                dual=False,
                                                                                                fit_intercept=True,
                                                     

In [56]:
import joblib
joblib.dump(model,'./automl_model.pkl')
print(best_remote_run.get_tags())

{'_aml_system_azureml.automlComponent': 'AutoML', '_aml_system_ComputeTargetStatus': '{"AllocationState":"steady","PreparingNodeCount":0,"RunningNodeCount":4,"CurrentNodeCount":4}', 'mlflow.source.type': 'JOB', 'mlflow.source.name': 'automl_driver.py', '_aml_system_codegen': 'completed', '_aml_system_automl_is_child_run_end_telemetry_event_logged': 'True'}


In [57]:
from azureml.core.model import Model

model = Model.register(workspace = ws,
                        model_path ="./automl_model.pkl",
                        model_name = "automl")

Registering model automl


In [58]:
%%writefile amlscore.py
import json
import numpy as np
import os
import pickle
import joblib
import pandas as pd
from azureml.core.model import Model

def init():
    global model
    model_path = Model.get_model_path("automl")
    model = joblib.load(model_path)

def run(raw_data):
    try:
        data = json.loads(raw_data)['data']
        data = pd.DataFrame.from_dict(data)
        # make prediction
        result = model.predict(data)
        return result.tolist()
    except Exception as ex:
        error = str(ex)
        return error

Overwriting amlscore.py


In [59]:
from azureml.core.environment import Environment
from azureml.core.model import InferenceConfig
from azureml.core.webservice import AciWebservice, Webservice
from azureml.core.model import Model
from azureml.automl.core.shared import constants
from azureml.core.conda_dependencies import CondaDependencies

best_remote_run.download_file(constants.CONDA_ENV_FILE_PATH, "myenv.yml")
myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")

inference_config = InferenceConfig(entry_script='amlscore.py', environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, memory_gb=1, description="heart-failure-model-final-one")

service = Model.deploy(ws, "automl-final", [model], inference_config, aciconfig)

service.wait_for_deployment(True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2023-05-28 17:58:44+00:00 Creating Container Registry if not exists.
2023-05-28 17:58:44+00:00 Registering the environment.
2023-05-28 17:58:45+00:00 Use the existing image.
2023-05-28 17:58:45+00:00 Generating deployment configuration.
2023-05-28 17:58:46+00:00 Submitting deployment to compute.
2023-05-28 17:58:53+00:00 Checking the status of deployment automl-final..
2023-05-28 18:01:08+00:00 Checking the status of inference endpoint automl-final.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [60]:
print(service.get_logs())

/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-28T18:00:40,514562500+00:00 - rsyslog/run 
2023-05-28T18:00:40,514512200+00:00 - iot-server/run 
bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by bash)
2023-05-28T18:00:40,534078500+00:00 - gunicorn/run 
2023-05-28T18:00:40,540021800+00:00 | gunicorn/run | 
2023-05-28T18:00:40,541884500+00:00 | gunicorn/run | ####################################

In [61]:
scoring_uri = service.scoring_uri

print(f'\nservice state: {service.state}\n')
print(f'scoring URI: \n{service.scoring_uri}\n')
print(f'swagger URI: \n{service.swagger_uri}\n')

print(service.scoring_uri)
print(service.swagger_uri)



service state: Healthy

scoring URI: 
http://124c4c0b-7a11-42d7-942a-efecf89bd7b2.westeurope.azurecontainer.io/score

swagger URI: 
http://124c4c0b-7a11-42d7-942a-efecf89bd7b2.westeurope.azurecontainer.io/swagger.json

http://124c4c0b-7a11-42d7-942a-efecf89bd7b2.westeurope.azurecontainer.io/score
http://124c4c0b-7a11-42d7-942a-efecf89bd7b2.westeurope.azurecontainer.io/swagger.json


In [85]:
import requests
import json

data={"data":
  [{'age': 49.0,
  'anaemia': 1,
  'creatinine_phosphokinase': 80,
  'diabetes': 1,
  'ejection_fraction': 30,
  'high_blood_pressure': 1,
  'platelets': 427000,
  'serum_creatinine': 1,
  'serum_sodium': 138,
  'sex': 0,
  'smoking': 0,
  'time': 12}]
  }

input_data=json.dumps(data)

with open("data.json","w") as file:
    file.write(input_data)

headers={"Content-Type":"application/json"}

result=requests.post(scoring_uri, input_data, headers=headers)
print("Status code: ",result.status_code)
print("The output is: ",result.json())



Status code:  200
The output is:  [1]


In [86]:
import urllib.request
import json
import os
import ssl

def allowSelfSignedHttps(allowed):
    # bypass the server certificate verification on client side
    if allowed and not os.environ.get('PYTHONHTTPSVERIFY', '') and getattr(ssl, '_create_unverified_context', None):
        ssl._create_default_https_context = ssl._create_unverified_context

allowSelfSignedHttps(True) # this line is needed if you use self-signed certificate in your scoring service.

# Request data goes here
# The example below assumes JSON formatting which may be updated
# depending on the format your endpoint expects.
# More information can be found here:
# https://docs.microsoft.com/azure/machine-learning/how-to-deploy-advanced-entry-script
data={"data":
  [{'age': 49.0,
  'anaemia': 1,
  'creatinine_phosphokinase': 80,
  'diabetes': 1,
  'ejection_fraction': 30,
  'high_blood_pressure': 1,
  'platelets': 427000,
  'serum_creatinine': 1,
  'serum_sodium': 138,
  'sex': 1,
  'smoking': 0,
  'time': 12}]
  }

body = str.encode(json.dumps(data))

url = 'http://124c4c0b-7a11-42d7-942a-efecf89bd7b2.westeurope.azurecontainer.io/score'


headers = {'Content-Type':'application/json'}

req = urllib.request.Request(url, body, headers)

try:
    response = urllib.request.urlopen(req)

    result = response.read()
    print("The output is: ", result)
except urllib.error.HTTPError as error:
    print("The request failed with status code: " + str(error.code))

The output is:  b'[1]'


In [63]:
result.status_code

200

TODO: In the cell below, print the logs of the web service and delete the service

In [83]:
#logs of the webservice
logs=service.get_logs()
print(logs)

/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
/bin/bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by /bin/bash)
2023-05-28T18:00:40,514562500+00:00 - rsyslog/run 
2023-05-28T18:00:40,514512200+00:00 - iot-server/run 
bash: /azureml-envs/azureml_c9289db7d7cee531fa8170c6d853612d/lib/libtinfo.so.6: no version information available (required by bash)
2023-05-28T18:00:40,534078500+00:00 - gunicorn/run 
2023-05-28T18:00:40,540021800+00:00 | gunicorn/run | 
2023-05-28T18:00:40,541884500+00:00 | gunicorn/run | ####################################

In [65]:
#delete the deployed webservice
#service.delete()